# Chương 6: Custom API trong TensorFlow và thí nghiệm xây dựng mô hình

# 1) Custom Layers (Lớp tùy chỉnh)
Nếu một loại network layer chưa được định nghĩa sẵn trong TensorFlow, ta có thể tạo một custom layer mới bằng cách kế thừa lớp tf.keras.layers.Layer. Tham khảo thêm tại trang API của TensorFlow về layers.

Ví dụ dưới đây minh họa cách tạo một custom convolutional layer (Conv2D):

In [2]:
!pip install tensorflow
import tensorflow as tf

In [3]:
class CustomConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding="VALID", **kwargs):
        super(CustomConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = (1, *strides, 1)
        self.padding = padding

    def build(self, input_shape):
        kernel_h, kernel_w = self.kernel_size
        input_dim = input_shape[-1]
        # weights
        self.w = self.add_weight(name='kernel',
                                 shape=(kernel_h, kernel_w, input_dim, self.filters),
                                 initializer='glorot_uniform',  # weight initialization
                                 trainable=True)
        # bias
        self.b = self.add_weight(name='bias',
                                 shape=(self.filters,),
                                 initializer='zeros',  # bias initialization
                                 trainable=True)

    def call(self, inputs):
        x = tf.nn.conv2d(inputs, self.w, self.strides, padding=self.padding) # Convolution operation
        x = tf.nn.bias_add(x, self.b)
        x = tf.nn.relu(x)  # using Relu activation function
        return x

# 2) Custom Loss (Hàm mất mát tùy chỉnh)

Các loss function được TensorFlow cung cấp sẵn không thể bao phủ mọi bài toán. Vì vậy, nếu không có loss phù hợp để giải quyết bài toán cụ thể, ta cần tự định nghĩa custom loss function.

Ví dụ dưới đây tạo một hàm custom categorical crossentropy sử dụng logits:

In [4]:
def custom_categorical_crossentropy(y_true, y_pred):
    # x = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_pred), reduction_indices=[1]))
    x = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return x


# 3) Custom Metrics (Chỉ số đánh giá tùy chỉnh)

Nếu một metric chưa có sẵn trong TensorFlow, có thể tạo metric mới bằng cách kế thừa lớp tf.keras.metrics.Metric.

Ví dụ dưới đây tạo CustomCategoricalAccuracy:

In [25]:
class CustomCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_catrgorical_accuracy', **kwargs):
        super(CustomCategoricalAccuracy, self).__init__(name=name, **kwargs)
        # the number of correct predictions
        self.correct = self.add_weight(name='correct_numbers', shape=(), initializer='zeros')
        # the amount of all data
        self.total = self.add_weight(name='total_numbers', shape=(), initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # (y_true is represented by using One-hot encoding)
        # get the index with the largest value
        y_true = tf.argmax(y_true, axis=-1)
        # get the index with the largest value
        y_pred = tf.argmax(y_pred, axis=-1)
        # Compare whether the prediction result is correct, true will return True (correct),false will return False (error)
        values = tf.equal(y_true, y_pred)
        # Convert to floating point: True (correct) = 1.0, False (false) = 0.0
        values = tf.cast(values, tf.float32)
        # Computes the sum of elements
        values_sum = tf.reduce_sum(values)
        num_values = tf.cast(tf.size(values), tf.float32)
        self.correct.assign_add(values_sum)  # Update the total number of correct predictions

        self.total.assign_add(num_values)  # Total amount of updated data

    def result(self):
        # Calculate accuracy
        return tf.math.divide_no_nan(self.correct, self.total)

    def reset_states(self):
        # Variables will be reinitialized after each Epoch
        self.correct.assign(0.)
        self.total.assign(0.)

# 4) Custom Callbacks (Callback tùy chỉnh)

Nếu TensorFlow chưa cung cấp callback phù hợp, có thể tạo callback mới bằng cách kế thừa tf.keras.callbacks.Callback.

Ví dụ dưới đây là callback lưu mô hình (hoặc weights) khi metric theo dõi được cải thiện:

In [21]:
class SaveModel(tf.keras.callbacks.Callback):
    def __init__(self, weights_file, monitor='loss', mode='min', save_weights_only=False):
        super(SaveModel, self).__init__()
        self.weights_file = weights_file
        self.monitor = monitor
        self.mode = mode
        self.save_weights_only = save_weights_only
        if mode == 'min':
            self.best = np.inf
        else:
            self.best = -np.inf

    def save_model(self):
        if self.save_weights_only:
            self.model.save_weights(self.weights_file)
        else:
            self.model.save(self.weights_file)

    def on_epoch_end(self, epoch, logs=None):
        monitor_value = logs.get(self.monitor)
        if self.mode == 'min' and monitor_value < self.best:
            self.save_model()
            self.best = monitor_value
        elif self.mode == 'max' and monitor_value > self.best:
            self.save_model()
            self.best = monitor_value

# 5) Thí nghiệm: triển khai 2 mô hình bằng Keras API và Custom API

Có hai kịch bản xây dựng mạng nơ-ron cho bài toán phân loại đa lớp:

Model-1: dùng high-level Keras API.

Model-2: dùng custom API (Custom Layer/Loss/Metric/Callback).

Hai mô hình được huấn luyện riêng trên CIFAR-10 để so sánh hiệu năng.



# (1) Chuẩn bị dữ liệu

Import packages:

In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers

# --- Inlined functions from preprocessing.py to fix import error ---
IMAGE_SIZE = 32
NUM_CLASSES = 10 # For CIFAR-10

def parse_fn(features):
    image = features['image']
    label = features['label']
    # Normalize image to [0, 1] and resize
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    # One-hot encode the label
    label = tf.one_hot(label, depth=NUM_CLASSES)
    return image, label

def parse_aug_fn(features):
    image, label = parse_fn(features) # Call parse_fn to get processed image and label
    # Apply random horizontal flip
    image = tf.image.random_flip_left_right(image)
    # Apply random brightness adjustment
    image = tf.image.random_brightness(image, max_delta=0.2)
    # Apply random contrast adjustment
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image, label
# --- End of inlined functions ---

Tải dữ liệu CIFAR-10:

In [8]:
# Divide training data 1: 9 (1 part for validation and 9 parts for training)
train_split, valid_split = ['train[:90%]', 'train[90%:]']
# get the training data
train_data, info = tfds.load("cifar10", split=train_split, with_info=True)
# get the valid data
valid_data = tfds.load("cifar10", split=valid_split)
# get the test data
test_data = tfds.load("cifar10", split=tfds.Split.TEST)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.SDA8DF_3.0.2/cifar10-train.tfrecord*...:   0%|         …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.SDA8DF_3.0.2/cifar10-test.tfrecord*...:   0%|          …

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


Thiết lập pipeline dữ liệu:

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE  # Automatic adjustment mode
batch_size = 64  # Batch size

# Moved data loading from TFD-HOC37waJ to ensure variables are defined
train_split, valid_split = ['train[:90%]', 'train[90%:]']
train_data, info = tfds.load("cifar10", split=train_split, with_info=True)
valid_data = tfds.load("cifar10", split=valid_split)
test_data = tfds.load("cifar10", split=tfds.Split.TEST)

train_num = int(info.splits['train'].num_examples / 10) * 9  # Number of training data

train_data = train_data.shuffle(train_num)  # Shuffle the training data
# Training data
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
# Set batch size and turn on prefetch mode
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# Validation data
valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# Set batch size and turn on prefetch mode
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# Test data
test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
# #Set batch size and turn on prefetch mode
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# Diagnostic: Print shapes of one batch
for images, labels in train_data.take(1):
    print("Shape of images batch:", images.shape)
    print("Shape of labels batch:", labels.shape)


Shape of images batch: (64, 32, 32, 3)
Shape of labels batch: (64, 10)


# 2. Xây dựng và huấn luyện mô hình


# a) Model-1: Sử dụng high-level Keras API của TensorFlow
Kiến trúc Model-1

Input layer shape: (32, 32, 3)

5 convolutional layers, sau mỗi lớp là ReLU

1 max pooling layer

1 flatten layer để chuyển tensor về 1 chiều

1 fully connected layer

1 dropout layer với discard rate = 50%

Output fully connected layer có 10 neurons, sau đó là softmax function
(Lưu ý: trong code output là Dense(10) và compile loss dùng from_logits=True)

Xây dựng mạng

In [10]:
#building network
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(inputs)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(256, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

# Create model
model_1 = keras.Model(inputs, outputs, name='model-1')
model_1.summary()


Model: "model-1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 30, 30, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 11, 11, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 9, 9, 128)      │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 64)       │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 941,066 (3.59 MB)

 Trainable params: 941,066 (3.59 MB)

 Non-trainable params: 0 (0.00 B)

Tạo callback

Lưu training log:

In [11]:
# Save training log
logs_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dir='lab6-logs')


Tạo thư mục lưu model và callback checkpoint:

In [12]:
# create a path to save models
model_dirs = logs_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)
save_model = tf.keras.callbacks.ModelCheckpoint(model_dirs + '/save.h5',
                                                monitor='val_catrgorical_accuracy',
                                                mode='max')


Thiết lập optimizer, loss, metric

In [13]:
model_1.compile(keras.optimizers.Adam(),
                loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[keras.metrics.CategoricalAccuracy()])


Huấn luyện Model-1

In [14]:
model_1.fit(train_data,
            epochs=100,
            validation_data=valid_data,
            callbacks=[model_cbk, save_model])

Epoch 1/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - categorical_accuracy: 0.2041 - loss: 2.0879

704/704 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - categorical_accuracy: 0.2042 - loss: 2.0877 - val_categorical_accuracy: 0.4032 - val_loss: 1.6599
Epoch 2/100
699/704 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - categorical_accuracy: 0.3763 - loss: 1.6687

704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - categorical_accuracy: 0.3766 - loss: 1.6682 - val_categorical_accuracy: 0.5098 - val_loss: 1.3593
Epoch 3/100
702/704 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - categorical_accuracy: 0.4754 - loss: 1.4568

704/704 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - categorical_accuracy: 0.4755 - loss: 1.4566 - val_categorical_accuracy: 0.5506 - val_loss: 1.2341
Epoch 4/100
700/704 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - categorical_accuracy: 0.5308 - loss: 1.3131

704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - categorical_accuracy: 0.5309 - loss: 1.3129 - val_categorical_accuracy: 0.6194 - val_loss: 1.0529
Epoch 5/100
701/704 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - categorical_accuracy: 0.5817 - loss: 1.1982

...

704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - categorical_accuracy: 0.9426 - loss: 0.1758 - val_categorical_accuracy: 0.7664 - val_loss: 1.4506
Epoch 97/100
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - categorical_accuracy: 0.9400 - loss: 0.1992

704/704 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - categorical_accuracy: 0.9400 - loss: 0.1991 - val_categorical_accuracy: 0.7716 - val_loss: 1.6136
Epoch 98/100
702/704 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - categorical_accuracy: 0.9488 - loss: 0.1663

704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - categorical_accuracy: 0.9488 - loss: 0.1663 - val_categorical_accuracy: 0.7650 - val_loss: 1.4577
Epoch 99/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - categorical_accuracy: 0.9523 - loss: 0.1490

704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - categorical_accuracy: 0.9523 - loss: 0.1490 - val_categorical_accuracy: 0.7684 - val_loss: 1.5362
Epoch 100/100
700/704 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - categorical_accuracy: 0.9496 - loss: 0.1572

704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - categorical_accuracy: 0.9495 - loss: 0.1573 - val_categorical_accuracy: 0.7622 - val_loss: 1.5072


In [15]:
%%writefile /content/preprocessing.py
import tensorflow as tf

IMAGE_SIZE = 32
NUM_CLASSES = 10 # For CIFAR-10

def parse_fn(features):
    image = features['image']
    label = features['label']
    # Normalize image to [0, 1] and resize
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    # One-hot encode the label
    label = tf.one_hot(label, depth=NUM_CLASSES)
    return image, label

def parse_aug_fn(features):
    image, label = parse_fn(features) # Call parse_fn to get processed image and label
    # Apply random horizontal flip
    image = tf.image.random_flip_left_right(image)
    # Apply random brightness adjustment
    image = tf.image.random_brightness(image, max_delta=0.2)
    # Apply random contrast adjustment
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image, label

Overwriting /content/preprocessing.py


# b) Model-2: Sử dụng custom API của TensorFlow

Kiến trúc Model-2

Input layer shape: (32, 32, 3)

5 convolutional layers, sau mỗi lớp là ReLU

1 max pooling layer

1 flatten layer

1 fully connected layer

1 dropout layer discard rate 50%

Output fully connected layer có 10 neurons, sau đó là softmax function

Custom Layer, Custom Loss, Custom Metrics, Custom Callbacks được dùng để tạo và huấn luyện Model-2.
(Cần chạy phần định nghĩa custom ở trên trước khi build Model-2.)

Xây dựng Model-2

In [16]:
# Building Model-2
inputs = keras.Input(shape=(32, 32, 3))
x = CustomConv2D(64, (3, 3))(inputs)
x = layers.MaxPool2D()(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(256, (3, 3))(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(64, (3, 3))(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

# Creating Model-2
model_2 = keras.Model(inputs, outputs, name='model-2')
model_2.summary()


Model: "model-2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_conv2d (CustomConv2D)    │ (None, 30, 30, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_conv2d_1 (CustomConv2D)  │ (None, 13, 13, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_conv2d_2 (CustomConv2D)  │ (None, 11, 11, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_conv2d_3 (CustomConv2D)  │ (None, 9, 9, 128)      │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_conv2d_4 (CustomConv2D)  │ (None, 7, 7, 64)       │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 941,066 (3.59 MB)

 Trainable params: 941,066 (3.59 MB)

 Non-trainable params: 0 (0.00 B)

Tạo callback: dùng custom callback để lưu weights

Lưu log TensorBoard:

In [17]:
# Save training log
logs_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dir='lab6-logs')


Tạo thư mục lưu model:

In [18]:
# create a storage path
model_dirs = logs_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)


Custom callback lưu weights tốt nhất:

In [29]:
custom_save_model = SaveModel(model_dirs + '/custom_save.weights.h5',
                              monitor='val_custom_catrgorical_accuracy',
                              mode='max',
                              save_weights_only=True)

Thiết lập optimizer, loss, metric

In [30]:
model_2.compile(keras.optimizers.Adam(),
           loss=custom_categorical_crossentropy,
           metrics=[CustomCategoricalAccuracy()])

Huấn luyện Model-2

In [31]:
model_2.fit(train_data,
            epochs=100,
            validation_data=valid_data,
            callbacks=[model_cbk, custom_save_model])

Epoch 1/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - custom_catrgorical_accuracy: 0.4407 - loss: 1.5419 - val_custom_catrgorical_accuracy: 0.5490 - val_loss: 1.2553
Epoch 2/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - custom_catrgorical_accuracy: 0.5036 - loss: 1.3896 - val_custom_catrgorical_accuracy: 0.5612 - val_loss: 1.2409
Epoch 3/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - custom_catrgorical_accuracy: 0.5408 - loss: 1.2909 - val_custom_catrgorical_accuracy: 0.6216 - val_loss: 1.0501
Epoch 4/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - custom_catrgorical_accuracy: 0.5688 - loss: 1.2095 - val_custom_catrgorical_accuracy: 0.6382 - val_loss: 1.0258
Epoch 5/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - custom_catrgorical_accuracy: 0.6016 - loss: 1.1372 - val_custom_catrgorical_accuracy: 0.6496 - val_loss: 0.9903
Epoch 6/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - custom_catrgorical_accuracy: 0.6170 - loss: 1.0926 - val_custom_catrgorical_accuracy: 0.6556 - val_los

# 3. Hiển thị kết quả (Displaying results)
Load weights của Model-1 và Model-2

In [33]:
model_1.load_weights(model_dirs+'/save.h5')
model_2.load_weights(model_dirs+'/custom_save.weights.h5')

Đánh giá trên test data

In [34]:
loss_1, acc_1 = model_1.evaluate(test_data)
loss_2, acc_2 = model_2.evaluate(test_data)
loss = [loss_1, loss_2]
acc = [acc_1, acc_2]
dict = {"Loss": loss, "Accuracy": acc}
pd.DataFrame(dict)


157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - categorical_accuracy: 0.7671 - loss: 1.4487
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - custom_catrgorical_accuracy: 0.7666 - loss: 1.0298


,Loss,Accuracy
0,1.503094,0.7628
1,1.028189,0.7669
